# Imports

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re

In [14]:
data = {
    'raw_message': [
        'hello',
        'hi', # for some reason this triggers really large values in things like 'bio' and 'cognitive mech'
        'hi!',
        'I am happy today', # first_person is always 0
        'i am good', # first_person is always 0
        '()()()()', # all symbols leads to the regex returning len(regex),
        ':-|', # another all symbol one
        'I think perhaps this is maybe fine', # hedge words
        'who what when where why',
        'I am good happy wonderful great excellent', # lots of positive words
        'under the sea and above the waves', # prepositions,
        'i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?', # TODO - weirdly, the outputs in the real thing differ from the outputs in the test here...
        'i was conflicted because i could understand his frustration however i feel he should have maybe discussed strategies with how to approach the mother in law with his wife first.'
    ]
}

def preprocess_text(text):
  	# For each individual message: preprocess to remove anything that is not an alphabet or number from the string
	return(re.sub(r"[^a-zA-Z0-9 ]+", '',text).lower())

chat_df = pd.DataFrame(data)

chat_df["message"] = chat_df["raw_message"].apply(preprocess_text)

In [15]:
chat_df

,raw_message,message
0,hello,hello
1,hi,hi
2,hi!,hi
3,I am happy today,i am happy today
4,i am good,i am good
5,()()()(),
6,:-|,
7,I think perhaps this is maybe fine,i think perhaps this is maybe fine
8,who what when where why,who what when where why
9,I am good happy wonderful great excellent,i am good happy wonderful great excellent


In [16]:
with open("../../features/lexicons_dict.pkl", "rb") as lexicons_pickle_file:
    lexicons_dict = pickle.load(lexicons_pickle_file)

In [17]:
test_df = pd.concat(
		# Finding the # of occurances of lexicons of each type for all the messages.
		[pd.DataFrame(chat_df["message"].apply(lambda chat: len(re.findall(regex, chat))))\
			  							.rename({"message": lexicon_type}, axis=1)\
			for lexicon_type, regex in lexicons_dict.items()], 
		axis=1
	)

In [18]:
test_df

,discrepancies,hear,home,conjunction,certainty,inclusive,bio,achievement,adverbs,anxiety,...,auxiliary_verbs,cognitive_mech,preposition,first_person_plural,percept,second_person,positive_words,first_person,nltk_english_stopwords,hedge_words
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,2,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,2,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,2,0,...,1,3,0,0,0,0,1,1,3,1
8,0,0,0,1,0,0,0,0,2,0,...,0,1,0,0,0,0,0,0,5,0
9,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,5,1,2,0


A few potential error cases below --- it seems we are returning more positives than necessary!

In [19]:
# seems a bunch of false positives in the first person
re.findall(lexicons_dict["first_person"], "i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?")

['i', 'i']

In [20]:
re.findall(lexicons_dict["nltk_english_stopwords"], "i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?")

['i',
 'can',
 'how',
 'the',
 'is',
 'because',
 'they',
 'the',
 'was',
 'but',
 'i',
 'can',
 'the',
 's',
 'why',
 'should',
 'he',
 'have',
 'to',
 'as',
 'for',
 'his',
 'in']

In [21]:
re.findall(lexicons_dict["cognitive_mech"], "i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?")

['how',
 'because',
 'feel',
 'but',
 'understand',
 'feelings',
 'why',
 'should',
 'interpreter',
 'law']

In [22]:
re.findall(lexicons_dict["preposition"], "i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?")

['to', 'as', 'for', 'in']

In [29]:
re.findall(lexicons_dict["nltk_english_stopwords"], "i don't think he's an asshole for having the feelings he does, but i do think he is a bit of an asshole for handling it the way that he did without discussing it with his wife first or maybe finding a better way to approach it.")

['i',
 'don',
 't',
 'he',
 's',
 'an',
 'for',
 'having',
 'the',
 'he',
 'does',
 'but',
 'i',
 'do',
 'he',
 'is',
 'a',
 'of',
 'an',
 'for',
 'it',
 'the',
 'that',
 'he',
 'did',
 'it',
 'with',
 'his',
 'or',
 'a',
 'to',
 'it']

In [23]:
test_df.loc[4, :]

discrepancies             0
hear                      0
home                      0
conjunction               0
certainty                 0
inclusive                 0
bio                       0
achievement               0
adverbs                   0
anxiety                   0
third_person              0
negation                  0
swear                     0
death                     0
health                    0
see                       0
body                      0
family                    0
negative_affect           0
quantifier                0
positive_affect           1
insight                   0
humans                    0
present_tense             1
future_tense              0
past_tense                0
relative                  0
sexual                    0
inhibition                0
sadness                   0
social                    0
indefinite_pronoun        0
religion                  0
work                      0
money                     0
causation           

In [24]:
for lexicon_type, regex in lexicons_dict.items():
    print(lexicon_type)
    text = "hi"
    search_hits = re.findall(text, regex)
    print(search_hits)
    break

discrepancies
['hi']


In [25]:
regex

"\\bbesides\\b|\\bcould\\b|\\bcouldnt\\b|\\bcouldn't\\b|\\bcouldve\\b|\\bcould've\\b|\\bdesir\\S*\\b|\\bexpect\\S*\\b|\\bhope\\b|\\bhoped\\b|\\bhopeful\\b|\\bhopefully\\b|\\bhopefulness\\b|\\bhopes\\b|\\bhoping\\b|\\bideal\\S*\\b|\\bif\\b|\\bimpossib\\S*\\b|\\binadequa\\S*\\b|\\black\\S*\\b|\\bliabilit\\S*\\b|\\bmistak\\S*\\b|\\bmust\\b|\\bmustnt\\b|\\bmust'nt\\b|\\bmustn't\\b|\\bmustve\\b|\\bmust've\\b|\\bneed\\b|\\bneeded\\b|\\bneeding\\b|\\bneednt\\b|\\bneed'nt\\b|\\bneedn't\\b|\\bneeds\\b|\\bnormal\\b|\\bought\\b|\\boughta\\b|\\boughtnt\\b|\\bought'nt\\b|\\boughtn't\\b|\\boughtve\\b|\\bought've\\b|\\boutstanding\\b|\\bprefer\\S*\\b|\\bproblem\\S*\\b|\\brather\\b|\\bregardless\\b|\\bregret\\S*\\b|\\bshould\\b|\\bshouldnt\\b|\\bshould'nt\\b|\\bshouldn't\\b|\\bshoulds\\b|\\bshouldve\\b|\\bshould've\\b|\\bundesire\\S*\\b|\\bundo\\b|\\bunneccess\\S*\\b|\\bunneed\\S*\\b|\\bunwant\\S*\\b|\\bwanna\\b|\\bwant\\b|\\bwanted\\b|\\bwanting\\b|\\bwants\\b|\\bwish\\b|\\bwished\\b|\\bwishes\\b|\\b

In [26]:
re.match("hi", regex)

In [27]:
re.findall(regex, "hi")

[]